In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from wordcloud import WordCloud
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc

In [2]:
train = pd.read_csv('Train.csv')
valid= pd.read_csv('Valid.csv')
test = pd.read_csv('Test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [4]:
train.head(10)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
5,A terrible movie as everyone has said. What ma...,0
6,Finally watched this shocking movie last night...,1
7,I caught this film on AZN on cable. It sounded...,0
8,It may be the remake of 1987 Autumn's Tale aft...,1
9,My Super Ex Girlfriend turned out to be a plea...,1


In [5]:
def Preprocessing(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    text = [w for w in text.split(' ') if w not in stopwords.words('english')]
    text = [WordNetLemmatizer().lemmatize(token) for token in text]
    text = [WordNetLemmatizer().lemmatize(token,pos='v') for token in text]
    text = " ".join(text)
    return text

train['text'] = train.text.apply(lambda x:Preprocessing(x))
valid['text'] = valid.text.apply(lambda x:Preprocessing(x))
test['text']= test.text.apply(lambda x:Preprocessing(x))

KeyboardInterrupt: 

In [ ]:
print(train)

                                                    text  label
0      grow b 1965 watch love thunderbird mate school...      0
1      put movie dvd player sit coke chip expectation...      0
2      people know particular time past like feel nee...      0
3      even though great interest biblical movie bore...      0
4      im die hard dad army fan nothing ever change g...      1
...                                                  ...    ...
39995  western union something forget classic western...      1
39996  movie incredible piece work explore every nook...      1
39997  wife watch movie plan visit sicily stromboli s...      0
39998  first watch flatliners amaze necessary feature...      1
39999  would film good gross estimate 95000000 award ...      1

[40000 rows x 2 columns]


In [ ]:
train['label'].value_counts()

0    20019
1    19981
Name: label, dtype: int64

***Split Dataset***

In [ ]:
train_x = train['text']
valid_x = valid['text']
test_x = test['text']
train_y = train['label']
valid_y = valid['label']
test_y = test['label']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x)
train_x = tokenizer.texts_to_sequences(train_x)
valid_x = tokenizer.texts_to_sequences(valid_x)
test_x = tokenizer.texts_to_sequences(test_x)
train_x=pad_sequences(train_x,maxlen=120)
valid_x=pad_sequences(valid_x,maxlen=120)
test_x=pad_sequences(test_x,maxlen=120)

In [ ]:
size_of_vocabulary = len(tokenizer.word_index)+1
# print(size_of_vocabulary)

In [ ]:
print(train_x)

[[   0    0    0 ...  520 2439 4622]
 [ 218 7037  213 ...  884 5346  164]
 [   0    0    0 ...    3  409 4865]
 ...
 [   0    0    0 ... 1173 4174  503]
 [   0    0    0 ... 1876    1 2032]
 [   0    0    0 ...  169  332 6808]]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(size_of_vocabulary,128,input_length=120))
model.add(Bidirectional(LSTM(64,return_sequences=True,dropout=0.2)))
model.add(GlobalMaxPooling1D())
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)
model.summary()
# model = tf.keras.Sequential([tf.keras.layers.Embedding(size_of_vocabulary,128,input_length=120),
#                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
#                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#                              tf.keras.layers.Flatten(),
#                               tf.keras.layers.Dense(50,activation ="relu"),
#                              tf.keras.layers.Dense(1,activation = "softmax")])
# model.compile(loss = "binary_crossentropy",optimizer = "adam" , metrics = ['accuracy'])

# model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,embadding_dim,input_length = max_len),
#                              tf.keras.layers.Conv1D(50,5),
#                              tf.keras.layers.Flatten(),
#                              tf.keras.layers.Dense(100,activation = 'relu'),
#                              tf.keras.layers.Dense(1 ,activation = 'softmax')])
# model.compile(loss = "binary_crossentropy",optimizer = "adam" , metrics = ['accuracy'])
# model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 128)          18000768  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 120, 128)          98816     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 18,103,745
Trainable params: 18,103,745
Non-trainable params: 0
__________________________________________

In [ ]:
history = model.fit(train_x,train_y,batch_size=128,epochs=4,
                    validation_data=(valid_x,valid_y)
                    ,verbose=1,callbacks=[es,mc]
                   )


Epoch 1/4
313/313 [==============================] - 124s 383ms/step - loss: 0.3820 - accuracy: 0.8282 - val_loss: 0.2762 - val_accuracy: 0.8880

Epoch 00001: val_accuracy improved from -inf to 0.88800, saving model to best_model.h5
Epoch 2/4
313/313 [==============================] - 117s 373ms/step - loss: 0.1602 - accuracy: 0.9417 - val_loss: 0.3345 - val_accuracy: 0.8816

Epoch 00002: val_accuracy did not improve from 0.88800
Epoch 3/4
313/313 [==============================] - 115s 367ms/step - loss: 0.0717 - accuracy: 0.9759 - val_loss: 0.3679 - val_accuracy: 0.8770

Epoch 00003: val_accuracy did not improve from 0.88800
Epoch 4/4
313/313 [==============================] - 118s 376ms/step - loss: 0.0325 - accuracy: 0.9901 - val_loss: 0.5153 - val_accuracy: 0.8708

Epoch 00004: val_accuracy did not improve from 0.88800
Epoch 00004: early stopping


In [ ]:
#loading best model
model = load_model('best_model.h5')
# evaluate
loss,acc = model.evaluate(test_x,test_y)
print('Test Accuracy: {}%'.format(acc*100))

157/157 [==============================] - 8s 45ms/step - loss: 0.2792 - accuracy: 0.8878
Test Accuracy: 88.77999782562256%


ModuleNotFoundError: No module named 'iteration_utilities'

In [ ]:
# import os
# os.chdir(r'/kaggle/working')
xx = tokenizer.texts_to_sequences(['Half of this review will be me gushing about the action. Wow. Just wow. I was in complete awe. There were multiple times I was both tearing up and laughing at how unbelievably amazing the fight scenes are. Now this is nothing new for the John Wick series. But for me, this is far and away the best action of the entire series.Im an action junkie and am repeatedly disappointed with action movies and their terrible fight scenes, filled with shaky cam, quick cuts and close-up shots (looking at you Snake Eyes and Mortal Kombat). Then I watch a John Wick movie and feel so spoiled, while also being baffled at why other action movies cant learn from this. The fights are so fast-paced yet completely clear and easy to follow.'])
print(model.predict(xx))

[[0.3973202]]


In [ ]:
#perdict test
ypred = model.predict(test_x)
ypred[ypred>0.5]=1
ypred[ypred<=0.5]=0
#Confusion Metrics
print(confusion_matrix(test_y,ypred))

In [ ]:
print(classification_report(test_y,ypred))